In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import math

In [ ]:
def MyPerceptron (Data,N):
    m,n=Data.shape
    X=Data[:,0:n-1]
    X=np.append(X,np.ones([m,1]),axis=1)
    T=Data[:,n-1]
    w=np.append(1,np.zeros([n-1,]))
    L=np.ones([1,m])
    Miss=[]
    L=L[0]
    L=L.tolist()
    T=T.tolist()
    for i in range (0,m):
        if L[i] !=T[i]:
            Miss.append(i)
    Iter=0
    while (np.size(Miss)!=0 and Iter<5000):
        Iter=Iter+1
        w=w+N*np.transpose(X[Miss[0],:])*T[Miss[0]]
        Miss=[]
        for i in range (0,m):
            if np.matmul(np.transpose(w),np.transpose(X[i,:]))>0:
                L[i]=1
            else:
                L[i]=-1
            if L[i] !=T[i]:
                if np.isin(i,Miss)==0:
                    Miss.append(i)
                    break
    return w, Iter;
         

In [ ]:
N=[1,10**-1,10**-2,10**-3,10**-4,10**-5,10**-6,10**-7,10**-8,10**-9];
AccuracyAll=[];
for n in range (0,10):
    ## Step1: reading each image + appending to X array
    X=[];
    for i in range (1,2401):
        img=np.array(Image.open('Assignment 2 Dataset/Train/'+str(i)+'.jpg'))
        img=np.reshape(img,784)
        img=img.astype(float)
        X.append(img)
    ## Step2: creating a weight vector for each digit + appending to WAll array
    WAll=[]
    for i in range (0,10):
        T=-1*np.ones([240*10,1])
        T[i*240:(i+1)*240]=1
        s_train=np.append(X,T,axis=1)
        w, Iter=MyPerceptron(s_train,N[n])
        WAll.append(w)
    ## Step3: using weight vectors on Test images to predict a class for each
    C=[]
    for i in range (1,201):
        I = np.array(Image.open('Assignment 2 Dataset/Test/'+str(i)+'.jpg'))
        XT=np.reshape(I,(1,784))
        XT=XT.astype(float)
        XT=np.append(XT,1)
        PClass=[]
        for j in range (0,10):
            WTranspose=np.transpose(WAll[j])
            P=np.matmul(WTranspose,XT)
            PClass.append(P)
        b=np.argmax(PClass)
        C.append(b)
    ## Step 4: creating a confusion matrix for all w of this n    
    ConfusionMat=np.zeros([10,10])    
    for i in range (0,10):
        Ci=C[i*20:(i+1)*20]
        for j in range (0,10):  
            ConfusionMat[i,j]=Ci.count(j)
    Accuracy=np.diag(ConfusionMat)
    AccuracyAll.append(Accuracy)
    MeanAccuracy=np.average(Accuracy)
     
print(Accuracy)        

In [ ]:
print(Accuracy[0])   
print(Accuracy[1])   
print(Accuracy[2])   
print(Accuracy[3])  
print(Accuracy[4])   
print(Accuracy[5])   
print(Accuracy[6])   
print(Accuracy[7]) 
print(Accuracy[8])   
print(Accuracy[9])

In [ ]:
# Random Forest Algorithm on Sonar Dataset
from random import seed
from random import randrange
from csv import reader
from math import sqrt

# Load a CSV file
def load_csv(filename):
	dataset = list()
	with open(filename, 'r') as file:
		csv_reader = reader(file)
		for row in csv_reader:
			if not row:
				continue
			dataset.append(row)
	return dataset

# Convert string column to float
def str_column_to_float(dataset, column):
	for row in dataset:
		row[column] = float(row[column].strip())

# Convert string column to integer
def str_column_to_int(dataset, column):
	class_values = [row[column] for row in dataset]
	unique = set(class_values)
	lookup = dict()
	for i, value in enumerate(unique):
		lookup[value] = i
	for row in dataset:
		row[column] = lookup[row[column]]
	return lookup

# Split a dataset into k folds
def cross_validation_split(dataset, n_folds):
	dataset_split = list()
	dataset_copy = list(dataset)
	fold_size = int(len(dataset) / n_folds)
	for i in range(n_folds):
		fold = list()
		while len(fold) < fold_size:
			index = randrange(len(dataset_copy))
			fold.append(dataset_copy.pop(index))
		dataset_split.append(fold)
	return dataset_split

# Calculate accuracy percentage
def accuracy_metric(actual, predicted):
	correct = 0
	for i in range(len(actual)):
		if actual[i] == predicted[i]:
			correct += 1
	return correct / float(len(actual)) * 100.0

# Evaluate an algorithm using a cross validation split
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
	folds = cross_validation_split(dataset, n_folds)
	scores = list()
	for fold in folds:
		train_set = list(folds)
		train_set.remove(fold)
		train_set = sum(train_set, [])
		test_set = list()
		for row in fold:
			row_copy = list(row)
			test_set.append(row_copy)
			row_copy[-1] = None
		predicted = algorithm(train_set, test_set, *args)
		actual = [row[-1] for row in fold]
		accuracy = accuracy_metric(actual, predicted)
		scores.append(accuracy)
	return scores

# Split a dataset based on an attribute and an attribute value
def test_split(index, value, dataset):
	left, right = list(), list()
	for row in dataset:
		if row[index] < value:
			left.append(row)
		else:
			right.append(row)
	return left, right

# Calculate the Gini index for a split dataset
def gini_index(groups, classes):
	# count all samples at split point
	n_instances = float(sum([len(group) for group in groups]))
	# sum weighted Gini index for each group
	gini = 0.0
	for group in groups:
		size = float(len(group))
		# avoid divide by zero
		if size == 0:
			continue
		score = 0.0
		# score the group based on the score for each class
		for class_val in classes:
			p = [row[-1] for row in group].count(class_val) / size
			score += p * p
		# weight the group score by its relative size
		gini += (1.0 - score) * (size / n_instances)
	return gini

# Select the best split point for a dataset
def get_split(dataset, n_features):
	class_values = list(set(row[-1] for row in dataset))
	b_index, b_value, b_score, b_groups = 999, 999, 999, None
	features = list()
	while len(features) < n_features:
		index = randrange(len(dataset[0])-1)
		if index not in features:
			features.append(index)
	for index in features:
		for row in dataset:
			groups = test_split(index, row[index], dataset)
			gini = gini_index(groups, class_values)
			if gini < b_score:
				b_index, b_value, b_score, b_groups = index, row[index], gini, groups
	return {'index':b_index, 'value':b_value, 'groups':b_groups}

# Create a terminal node value
def to_terminal(group):
	outcomes = [row[-1] for row in group]
	return max(set(outcomes), key=outcomes.count)

# Create child splits for a node or make terminal
def split(node, max_depth, min_size, n_features, depth):
	left, right = node['groups']
	del(node['groups'])
	# check for a no split
	if not left or not right:
		node['left'] = node['right'] = to_terminal(left + right)
		return
	# check for max depth
	if depth >= max_depth:
		node['left'], node['right'] = to_terminal(left), to_terminal(right)
		return
	# process left child
	if len(left) <= min_size:
		node['left'] = to_terminal(left)
	else:
		node['left'] = get_split(left, n_features)
		split(node['left'], max_depth, min_size, n_features, depth+1)
	# process right child
	if len(right) <= min_size:
		node['right'] = to_terminal(right)
	else:
		node['right'] = get_split(right, n_features)
		split(node['right'], max_depth, min_size, n_features, depth+1)

# Build a decision tree
def build_tree(train, max_depth, min_size, n_features):
	root = get_split(train, n_features)
	split(root, max_depth, min_size, n_features, 1)
	return root

# Make a prediction with a decision tree
def predict(node, row):
	if row[node['index']] < node['value']:
		if isinstance(node['left'], dict):
			return predict(node['left'], row)
		else:
			return node['left']
	else:
		if isinstance(node['right'], dict):
			return predict(node['right'], row)
		else:
			return node['right']

# Create a random subsample from the dataset with replacement
def subsample(dataset, ratio):
	sample = list()
	n_sample = round(len(dataset) * ratio)
	while len(sample) < n_sample:
		index = randrange(len(dataset))
		sample.append(dataset[index])
	return sample

# Make a prediction with a list of bagged trees
def bagging_predict(trees, row):
	predictions = [predict(tree, row) for tree in trees]
	return max(set(predictions), key=predictions.count)

# Random Forest Algorithm
def random_forest(train, test, max_depth, min_size, sample_size, n_trees, n_features):
	trees = list()
	for i in range(n_trees):
		sample = subsample(train, sample_size)
		tree = build_tree(sample, max_depth, min_size, n_features)
		trees.append(tree)
	predictions = [bagging_predict(trees, row) for row in test]
	return(predictions)



# Test the random forest algorithm
seed(2)
# load and prepare data
filename = 'sonar.all-data.csv'
dataset = load_csv(filename)
# convert string attributes to integers
for i in range(0, len(dataset[0])-1):
	str_column_to_float(dataset, i)
# convert class column to integers
str_column_to_int(dataset, len(dataset[0])-1)
# evaluate algorithm
n_folds = 5
max_depth = 10
min_size = 1
sample_size = 1.0
n_features = int(sqrt(len(dataset[0])-1))
for n_trees in [1, 5, 10]:
	scores = evaluate_algorithm(dataset, random_forest, n_folds, max_depth, min_size, sample_size, n_trees, n_features)
	print('Trees: %d' % n_trees)
	print('Scores: %s' % scores)
	print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

In [ ]:
from collections import Counter

import numpy as np

from .decision_tree import DecisionTree


def bootstrap_sample(X, y):
    n_samples = X.shape[0]
    idxs = np.random.choice(n_samples, n_samples, replace=True)
    return X[idxs], y[idxs]


def most_common_label(y):
    counter = Counter(y)
    most_common = counter.most_common(1)[0][0]
    return most_common


class RandomForest:
    def __init__(self, n_trees=10, min_samples_split=2, max_depth=100, n_feats=None):
        self.n_trees = n_trees
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.n_feats = n_feats
        self.trees = []

    def fit(self, X, y):
        self.trees = []
        for _ in range(self.n_trees):
            tree = DecisionTree(
                min_samples_split=self.min_samples_split,
                max_depth=self.max_depth,
                n_feats=self.n_feats,
            )
            X_samp, y_samp = bootstrap_sample(X, y)
            tree.fit(X_samp, y_samp)
            self.trees.append(tree)

    def predict(self, X):
        tree_preds = np.array([tree.predict(X) for tree in self.trees])
        tree_preds = np.swapaxes(tree_preds, 0, 1)
        y_pred = [most_common_label(tree_pred) for tree_pred in tree_preds]
        return np.array(y_pred)


# Testing
if __name__ == "__main__":
    # Imports
    from sklearn import datasets
    from sklearn.model_selection import train_test_split

    def accuracy(y_true, y_pred):
        accuracy = np.sum(y_true == y_pred) / len(y_true)
        return accuracy

    data = datasets.load_breast_cancer()
    X = data.data
    y = data.target

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=1234
    )

    clf = RandomForest(n_trees=3, max_depth=10)

    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    acc = accuracy(y_test, y_pred)

    print("Accuracy:", acc)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from collections import Counter

import random
random.seed(10)

def subsample(x,y, n_sample):
    samplex = list()
    sampley = list()
    while len(samplex) < n_sample:
        index = randrange(len(x))
        samplex.append(x[index])
        sampley.append(y[index])

    return samplex,sampley


def bagging_predict(trees, row):
    predictions = [predict(tree, row) for tree in trees]
    return max(set(predictions), key=predictions.count)

def most_common_label(y):
    counter = Counter(y)
    most_common = counter.most_common(1)[0][0]
    return most_common

def bootstrap_sample(X, y,n_samples):
    idxs = np.random.choice(n_samples, n_samples, replace=True)
    for i in range(len(idxs)):
        X[i]=X[idxs[i]]
        y[i]=y[idxs[i]]
    return X, y 

# def predict(self, X):
#     tree_preds = np.array([tree.predict(X) for tree in self.trees])
#     tree_preds = np.swapaxes(tree_preds, 0, 1)
#     y_pred = [most_common_label(tree_pred) for tree_pred in tree_preds]
#     return np.array(y_pred)

def RandomForest(X,y,NumOfTrees,MaxDepth,NumOfSamples):
    trees = []
    for i in range(NumOfTrees):
#         X_samp, y_samp = bootstrap_sample(X, y,NumOfSamples)
        X_samp, y_samp = subsample(X,y, NumOfSamples)
        tree = DecisionTreeClassifier(max_depth=MaxDepth,random_state=10)
        tree.fit(X_samp, y_samp)
        trees.append(tree)
        predictions = [bagging_predict(trees, row) for row in y]
        return(predictions)
#         print (trees[i])
#         tree_preds = np.array([tree.predict(X) for tree in trees])
#         tree_preds = np.swapaxes(tree_preds, 0, 1)
#         y_pred = [most_common_label(tree_pred) for tree_pred in tree_preds]
#         return(y_pred)
        